----
<div style="display: flex; align-items: center;">
    <img src="https://developers.lseg.com/content/dam/devportal/icons/logo/lseg-logo.svg" width="20%" style="vertical-align: top;">
    <h1 style="margin-left: 20px;">Data Library for Python</h1>
</div>

----

## Delivery layer - Bulk - Client File Store - Tick History
This notebook demonstrates how to browse the Client File Store (CFS) of the LSEG Data Platform (RDP) using the Refintiv Data Library. The delivery.bulk.cfs module used in this example gives you an fine control on CFS operations. Unless you have a specific need, it is recommended to use the delivery.bulk.PackageManager instead. The Package manager provides you with a higher level API that manages for you the 3 types of activities generally invoved with bulk files (downloading files, populating databases, querying databases).

#### Learn more

To learn more about the LSEG Data Library for Python please join the LSEG Developer Community. By [registering](https://developers.refinitiv.com/iam/register) and [logging](https://developers.refinitiv.com/content/devportal/en_us/initCookie.html) into the LSEG Developer Community portal you will have free access to a number of learning materials 
 on the [LSEG Developer Portal](https://developers.lseg.com/en)

#### Getting Help and Support

If you have any questions regarding using the API, please post them on 
the [LSEG Data Q&A Forum](https://community.developers.refinitiv.com/spaces/321/index.html). 
The LSEG Developer Community will be happy to help. 

## Set the configuration file location
For a better ease of use, you have the option to set initialization parameters of the LSEG Data Library in the _lseg-data.config.json_ configuration file. This file must be located beside your notebook, in your user folder or in a folder defined by the _LD_LIB_CONFIG_PATH_ environment variable. The _LD_LIB_CONFIG_PATH_ environment variable is the option used by this series of examples. The following code sets this environment variable.      

## Some Imports to start with

In [1]:
import lseg.data as ld
import lseg.data.delivery.cfs as cfs

## Open the data session

The open_session() function creates and open sessions based on the information contained in the **lseg-data.config.json** configuration file. It is assumed this configuration file lives within the current working directory and defines the default session to match your specified credentials.

In [2]:
# Open default session
ld.open_session()

<lseg.data.session.Definition object at 0x1d53b549090 {name='rdp'}>

## Retrieve Tick History data

### Browse Tick History buckets

In [35]:
response = cfs.buckets.Definition('history').get_data()

buckets = response.data.buckets
for bucket in buckets:
    print("Bucket: ", bucket.name)
    print("\tFile sets: ")
    for file_set in bucket:
        if hasattr(file_set, 'name'):           # Ensure the file set exists
            print("\t\t{}\tID:{}".format(file_set.name, file_set.id))
            print("\t\t\tFiles: ")
            for file in file_set:
                print("\t\t\t\t{}\tSize (in Bytes):{}".format(file.filename, file.file_size_in_bytes))

Bucket:  TICKHISTORY_VBD_NO_EMBARGO
	File sets: 
		MUS_ref-data-cusip-sedol_2022-10-11_UNLIMITED	ID:4000-0028-9a0db078-b629-96e81f80a873
			Files: 
				MUS-2022-10-11-REF-Data-CUSIP-SEDOL-1-of-1.csv.gz	Size (in Bytes):3890
		ICX_normalised_2012-11-24_UNLIMITED	ID:4000-0037-263d473f-920d-7d0b0396d012
			Files: 
				ICX-2012-11-24-NORMALIZEDMP-Report-1-of-1.csv.gz	Size (in Bytes):1113
				ICX-2012-11-24-NORMALIZEDMP-Data-1-of-1.csv.gz	Size (in Bytes):449
		JKT_rawLL2_2023-07-29_UNLIMITED	ID:4000-003d-8165eb6f-907c-b2ef2f81de36
			Files: 
				JKT-2023-07-29-LEGACYLEVEL2-Data-1-of-1.csv.gz	Size (in Bytes):688216
				JKT-2023-07-29-LEGACYLEVEL2-Report-1-of-1.csv.gz	Size (in Bytes):107310
		AEX_rawLL2_2013-06-06_UNLIMITED	ID:4000-0044-52c2faf9-ae39-a9d3152f393f
			Files: 
				AEX-2013-06-06-LEGACYLEVEL2-Data-1-of-1.csv.gz	Size (in Bytes):521187651
				AEX-2013-06-06-LEGACYLEVEL2-Report-1-of-1.csv.gz	Size (in Bytes):192646
		FKA_ref-data-sedol_2022-05-22_UNLIMITED	ID:4000-005c-59cedd34-92c5-d

### Get file set details

In [36]:
fileset_id = '4000-0037-263d473f-920d-7d0b0396d012'
response = cfs.files.Definition(fileset_id).get_data()
response.data.df

href       description  \
0  https://api.refinitiv.com/file-store/v1/files/...  Merged data file   
1  https://api.refinitiv.com/file-store/v1/files/...  Merged data file   

                                md5               created  \
0  533d222b9033e1ea254c4beb63a50e30  2019-12-12T22:03:12Z   
1  c66c3b1749dc885e222d3953e522e9b8  2019-12-12T22:03:12Z   

                                     id fileType  \
0  438d-f6af-263f2c31-abfd-7dacc3c1c7f3     File   
1  4a93-8ece-24d9988f-8409-7d06502a2dbc     File   

                                           filename  \
0  ICX-2012-11-24-NORMALIZEDMP-Report-1-of-1.csv.gz   
1    ICX-2012-11-24-NORMALIZEDMP-Data-1-of-1.csv.gz   

                                     storageLocation              modified  \
0  {'url': 'https://s3-us-east-1.amazonaws.com/a2...  2019-12-12T22:03:12Z   
1  {'url': 'https://s3-us-east-1.amazonaws.com/a2...  2019-12-12T22:03:12Z   

                              filesetId  fileSizeInBytes  
0  4000-0037-263d473f-920d-7d0b0396d012             1113  
1  4000-0037-263d473f-920d-7d0b0396d012              449

### Download files
For the sake of the example, only files under 500KB are downloaded

In [37]:
for file in response.data.files:
    print('Downloading "{}"...'.format(file.filename))
    if file.file_size_in_bytes > 500000:
        print('\t...File skipped.  "{}" is too big for this example. Size (Bytes): {}'.format(file.filename, file.file_size_in_bytes))
    else:
        file_downloader = cfs.file_downloader.Definition(file).retrieve()
        file_downloader.download()
        print('\t...Done')
        print('Extracting "{}"...'.format(file.filename))
        file_downloader.extract()
        print('\t...Done')

	...Done
Extracting "ICX-2012-11-24-NORMALIZEDMP-Report-1-of-1.csv.gz"...
	...Done
	...Done
Extracting "ICX-2012-11-24-NORMALIZEDMP-Data-1-of-1.csv.gz"...
	...Done


## Close the session

In [38]:
ld.close_session()